In [1]:
from datetime import datetime
from strategy.stratfact import StratFact
from backtester.backtester import Backtester

In [2]:
start = datetime(2016,1,1)
end = datetime(2021,1,1)

In [3]:
# rp = StratFact.create_strat(start,end,"rolling_percent",params= {"timeframe":"daily"
#                 ,"requirement":5
#                 ,"days":7
#                 ,"value":True
#                 ,"currency":"crypto"})
# pr = StratFact.create_strat(start,end,"progress_report",params={"timeframe":"quarterly"
#                     ,"requirement":5})
# fp = StratFact.create_strat(start,end,"financial_predict",{})
s = StratFact.create_strat(start,end,"speculation",{})

In [4]:
# rpt = Backtester.equity_timeseries_backtest(start,end,5,rp)

In [5]:
# pr = Backtester.equity_timeseries_backtest(start,end,5,rp)

In [6]:
# fpb = Backtester.equity_timeseries_backtest(start,end,4,fp)

In [7]:
sb = Backtester.equity_timeseries_backtest(start,end,5,s)
sb

single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single posi

,date,adjClose,ticker,seat,sell_date,sell_price,timeframe,score_requirement,requirement,category_training_year,model_training_year,number_of_training_weeks,value
0,2016-03-28,26.160000,CTLT,0,2016-04-05,27.468000,weekly,70,5,4,2,14,True
1,2016-04-06,27.740000,CTLT,0,2016-04-13,29.127000,weekly,70,5,4,2,14,True
2,2016-04-18,635.350000,AMZN,0,2016-05-02,667.117500,weekly,70,5,4,2,14,True
3,2016-05-03,671.320000,AMZN,0,2016-05-11,704.886000,weekly,70,5,4,2,14,True
4,2016-05-12,23.800000,CTLT,0,2016-05-16,24.990000,weekly,70,5,4,2,14,True
5,2016-05-17,695.270000,AMZN,0,2016-07-06,730.033500,weekly,70,5,4,2,14,True
6,2016-09-20,25.220344,WY,0,2016-09-22,26.481362,weekly,70,5,4,2,14,True
7,2020-01-13,168.100000,ABMD,0,2020-01-15,176.505000,weekly,70,5,4,2,14,True
8,2020-02-03,48.034997,CAH,0,2020-02-06,50.436747,weekly,70,5,4,2,14,True
9,2020-02-24,51.330000,ETSY,0,2020-02-27,53.896500,weekly,70,5,4,2,14,True
